In [2]:
# ✅ 라이브러리 설치
!pip install kobert-transformers transformers datasets seqeval --quiet

# ✅ 전체 파이프라인 실행
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import ast
from datasets import Dataset
from transformers import BertForTokenClassification, Trainer, TrainingArguments
from kobert_transformers import get_tokenizer
from sklearn.metrics import precision_recall_fscore_support

# ✅ CSV 로드 (파일 업로드 먼저 해야 함)
df = pd.read_csv("distilled_masked_dataset.csv")  # ← 업로드 후 파일명에 맞게 수정

# ✅ 문자열 -> 리스트로 변환
df["tokens"] = df["tokens"].apply(ast.literal_eval)
df["ner_tags"] = df["ner_tags"].apply(ast.literal_eval)

# ✅ 라벨 정의 (단일 B-ENT 사용)
label_list = ["O", "B-ENT"]
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for i, l in enumerate(label_list)}

# ✅ Dataset 변환
dataset = Dataset.from_pandas(df)

# ✅ 토큰화 함수
tokenizer = get_tokenizer()

def tokenize_and_align_labels(example):
    tokens = example["tokens"]
    labels = example["ner_tags"]
    encoding = tokenizer(tokens, is_split_into_words=True, padding="max_length", truncation=True, return_tensors="pt")
    input_len = encoding["input_ids"].shape[1]
    aligned_labels = labels[:input_len] + [-100] * (input_len - len(labels))
    return {
        "input_ids": encoding["input_ids"][0],
        "attention_mask": encoding["attention_mask"][0],
        "labels": [label2id.get(t, 0) if t != -100 else -100 for t in aligned_labels]
    }

tokenized_dataset = dataset.map(tokenize_and_align_labels)

# ✅ 평가 함수
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=2)
    labels = p.label_ids
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_preds = [[id2label[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(preds, labels)]
    precision, recall, f1, _ = precision_recall_fscore_support(
        sum(true_labels, []), sum(true_preds, []), average="macro", zero_division=0
    )
    return {"precision": precision, "recall": recall, "f1": f1}

# ✅ 모델 & 학습 인자 설정
model = BertForTokenClassification.from_pretrained("monologg/kobert", num_labels=len(label_list))

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)

# ✅ 학습 및 평가
trainer.train()
metrics = trainer.evaluate()
print(metrics)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


{'eval_loss': 0.0011622211895883083, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 8.3796, 'eval_samples_per_second': 35.801, 'eval_steps_per_second': 4.535, 'epoch': 5.0}


In [3]:
# ✅ 라이브러리 불러오기
import re

# ✅ 테스트 문장들
test_sentences = [
    "최유리(40세 여성)은 2022년 3월에 분당서울대병원에서 알츠하이머 퇴원을 시작했다.",
    "2023년 9월, 50세 여성인 박서준은 대구파티마병원에서 간암 재활 후 퇴원했다.",
    "강다현(50세 여성)은 올해 1월에 부산의료원에서 심근경색 입원을 시작했다.",
    "강다현(62세 남성)는 폐렴으로 분당서울대병원에 2024년 6월 치료하였다.",
    "분당서울대병원에 작년 12월 방문한 김철수(60세 여성)은 폐렴로 치료 받음."
]

# ✅ 정규표현식 기반 민감정보 탐지 함수
def regex_ner(text):
    tokens = text.split()
    tags = []

    for token in tokens:
        if re.search(r"[가-힣]{2,4}", token) and "(" in token:  # 이름 패턴
            tags.append("B-PER")
        elif re.search(r"\d{1,3}세", token):  # 나이
            tags.append("B-AGE")
        elif re.search(r"(병원|의료원|의원)", token):  # 병원
            tags.append("B-ORG")
        elif re.search(r"(당뇨|고혈압|암|치매|알츠하이머|심근경색|폐렴)", token):  # 질병명
            tags.append("B-DISEASE")
        else:
            tags.append("O")

    return tokens, tags

# ✅ 실행 및 결과 출력
for i, sent in enumerate(test_sentences):
    tokens, tags = regex_ner(sent)
    print(f"\n🔹 문장 {i+1}")
    print("문장:", sent)
    print("Tokens:", tokens)
    print("NER Tags:", tags)



🔹 문장 1
문장: 최유리(40세 여성)은 2022년 3월에 분당서울대병원에서 알츠하이머 퇴원을 시작했다.
Tokens: ['최유리(40세', '여성)은', '2022년', '3월에', '분당서울대병원에서', '알츠하이머', '퇴원을', '시작했다.']
NER Tags: ['B-PER', 'O', 'O', 'O', 'B-ORG', 'B-DISEASE', 'O', 'O']

🔹 문장 2
문장: 2023년 9월, 50세 여성인 박서준은 대구파티마병원에서 간암 재활 후 퇴원했다.
Tokens: ['2023년', '9월,', '50세', '여성인', '박서준은', '대구파티마병원에서', '간암', '재활', '후', '퇴원했다.']
NER Tags: ['O', 'O', 'B-AGE', 'O', 'O', 'B-ORG', 'B-DISEASE', 'O', 'O', 'O']

🔹 문장 3
문장: 강다현(50세 여성)은 올해 1월에 부산의료원에서 심근경색 입원을 시작했다.
Tokens: ['강다현(50세', '여성)은', '올해', '1월에', '부산의료원에서', '심근경색', '입원을', '시작했다.']
NER Tags: ['B-PER', 'O', 'O', 'O', 'B-ORG', 'B-DISEASE', 'O', 'O']

🔹 문장 4
문장: 강다현(62세 남성)는 폐렴으로 분당서울대병원에 2024년 6월 치료하였다.
Tokens: ['강다현(62세', '남성)는', '폐렴으로', '분당서울대병원에', '2024년', '6월', '치료하였다.']
NER Tags: ['B-PER', 'O', 'B-DISEASE', 'B-ORG', 'O', 'O', 'O']

🔹 문장 5
문장: 분당서울대병원에 작년 12월 방문한 김철수(60세 여성)은 폐렴로 치료 받음.
Tokens: ['분당서울대병원에', '작년', '12월', '방문한', '김철수(60세', '여성)은', '폐렴로', '치료', '받음.']
NER Tags: ['B-ORG', 'O', 'O', 'O', 'B-PER', 